## 参数管理
首先关注单隐藏层的MLP

In [5]:
import torch
from torch import nn
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.4494],
        [-0.4318]], grad_fn=<AddmmBackward0>)

## 参数访问

In [8]:
# 第2层的参数(不包含激活函数）
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0316,  0.0633, -0.2511,  0.0846,  0.1514, -0.2987, -0.1736, -0.1289]])), ('bias', tensor([-0.2216]))])


## 目标参数

In [10]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2216], requires_grad=True)
tensor([-0.2216])


In [13]:
# 还没有梯度
net[2].weight.grad == None

True

## 一次性访问所有参数

In [19]:
# 获取第1层的参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
# 获取全部网络的参数
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


## 访问最后一层的偏移 

In [18]:
net.state_dict()['2.bias'].data

tensor([-0.2216])

## 从嵌套块手机参数

In [21]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 给层命名
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.2814],
        [-0.2815]], grad_fn=<AddmmBackward0>)

## 我们已经设计了网络，看看它是如何组织的

In [23]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


## 内置初始化

In [26]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
# 对所有层遍历
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0035,  0.0003, -0.0087, -0.0043]), tensor(0.))

## 对某些块应用不同的初始化方法

In [30]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.1002, -0.4054,  0.1442,  0.0601])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


## 自定义初始化

In [32]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -0.0000, -0.0000, 8.4934],
        [0.0000, 0.0000, 0.0000, -0.0000]], grad_fn=<SliceBackward0>)

## 直接赋值 

In [34]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  2.0000,  2.0000, 10.4934])

## 参数绑定（不同层共享权重)

In [36]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))
net(X)

tensor([[-0.3528],
        [-0.3578]], grad_fn=<AddmmBackward0>)